In [13]:
# made by Gibeom LEE, HI LAB
# 한 폴더안에 모든 csv파일의 맨 처음부터 21번째 행까지를 지웁니다. (오실로스코프 정보)
# 또한, 시작하는 시간을 0초 부터 시작하게 합니다.
# 이미 처리된 파일일 경우, 작동하지 않습니다.

In [ ]:
import os
import re
import glob
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

In [15]:
# 파일이 있는 폴더 경로
folder_path = r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251019_데모 데이터\csv파일"


In [ ]:
MAX_WORKERS = min(8, (os.cpu_count() or 4))  # 병렬 처리 스레드 수
# ========================

# 콤마/탭/세미콜론 혼용 구분자 허용 + 불량 라인 스킵 + BOM 대응
READ_OPTS = dict(
    engine="python",
    sep=r"[,\t;]+",
    on_bad_lines="skip",
    encoding="utf-8-sig",
    header=None
)

# 숫자/구분자만 있는지 대략 판별(매우 빠름): 영문자/한글/따옴표 등 있으면 True
_non_numeric_re = re.compile(r"[A-Za-z가-힣\"']")

def fast_already_processed_check(path: str, sample_bytes: int = 4096) -> bool:
    """
    매우 빠르게 앞부분만 읽어서 '이미 처리된 파일(텍스트 헤더가 없음)'로 추정할지 판별.
    원래 로직의 '미리보기 contains_text'를 대체(더 빠름).
    True이면 '이미 처리됨'으로 간주하고 스킵.
    """
    try:
        with open(path, "rb") as f:
            head = f.read(sample_bytes)
        try:
            s = head.decode("utf-8-sig", errors="ignore")
        except Exception:
            s = head.decode(errors="ignore")
        # 영문/한글/따옴표 등 텍스트가 눈에 띄면 아직 미처리로 판단
        return _non_numeric_re.search(s) is None
    except Exception:
        # 문제가 있으면 미처리로 간주하여 처리 시도
        return False

def process_one_csv(file_path: str) -> str:
    try:
        # 파일명 공백 → 언더바
        dir_name, file_name = os.path.split(file_path)
        new_file_name = file_name.replace(' ', '_')
        new_file_path = os.path.join(dir_name, new_file_name)
        if new_file_name != file_name:
            os.replace(file_path, new_file_path)  # 같은 파일시스템 내 빠른 교체
            file_path = new_file_path
            print(f"[rename] {file_name} -> {new_file_name}")

        # 빠른 '이미 처리됨' 체크 (기존 preview 대체, 훨씬 빠름)
        if fast_already_processed_check(file_path):
            return f"[skip] Already processed (no header-like text): {file_path}"

        # 21행 스킵 + 유연한 구분자/불량라인 스킵으로 안전 파싱
        df = pd.read_csv(file_path, skiprows=21, **READ_OPTS)

        # 숫자 변환(1회) + NaN-only 행 제거
        df = df.apply(pd.to_numeric, errors="coerce").dropna(how="all")
        if df.empty:
            return f"[empty] No valid data in {file_path}"

        # 시간 재계산 (첫 열만 등간격으로 교체, 나머지 열 보존) — 원본 기능 유지
        num_rows, num_cols = df.shape
        if num_rows > 1 and num_cols >= 1:
            start_time = df.iat[0, 0]
            end_time   = df.iat[-1, 0]
            if pd.isna(start_time) or pd.isna(end_time):
                dt = 1.0
            else:
                dt = (end_time - start_time) / (num_rows - 1)
            df.iloc[:, 0] = np.arange(num_rows, dtype=float) * float(dt)

        # 저장 (일반적으로 to_csv가 savetxt보다 빠름)
        df.to_csv(file_path, index=False, header=False, float_format="%.10f")
        return f"[ok] {file_path}"

    except Exception as e:
        return f"[err] {file_path}: {e}"

def main():
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    if not csv_files:
        print("[info] No CSV files found.")
        return

    print(f"[info] Found {len(csv_files)} CSV files. Processing with {MAX_WORKERS} threads...")
    results = []
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futs = [ex.submit(process_one_csv, p) for p in csv_files]
        for fut in as_completed(futs):
            results.append(fut.result())

    # 로그 출력
    for msg in results:
        print(msg)

if __name__ == "__main__":
    main()


Error processing C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251019_데모 데이터\csv파일\Aloe_DEG_1.csv: Error tokenizing data. C error: Expected 2 fields in line 4, saw 5

Error processing C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251019_데모 데이터\csv파일\Aloe_DEG_2.csv: Error tokenizing data. C error: Expected 2 fields in line 4, saw 5

Error processing C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251019_데모 데이터\csv파일\Aloe_DEG_3.csv: Error tokenizing data. C error: Expected 2 fields in line 4, saw 5

Error processing C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251019_데모 데이터\csv파일\Aloe_DEG_4.csv: Error tokenizing data. C error: Expected 2 fields in line 4, saw 5

Skipped (이미 처리된 파일입니다.): C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0.

C:\Users\user\AppData\Local\Temp\ipykernel_14832\1943957492.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_14832\1943957492.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_14832\1943957492.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_14832\1943957492.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_14832\1943957492.py:19: FutureWarning: DataFrame.appl